In [147]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('data_for_spb_hakaton_entities1-Table 1.csv', sep=';')
history = pd.read_csv('history-Table 1.csv', sep=';', index_col=None)
sprints = pd.read_csv('sprints-Table 1.csv', sep=';', index_col=None)

In [148]:
# Сдвиг колонок влево, чтобы "entity_id" стало первой колонкой
history = history.reset_index()  # Сброс индекса
columns = history.columns.tolist()  # Получаем текущий список колонок
columns = columns[1:] + columns[:1]  # Перемещаем первую колонку в конец
history.columns = columns  # Переупорядочиваем названия колонок
if 'index' in history.columns:
    history.drop(columns=['index'], inplace=True)

In [149]:
history

,entity_id,history_property_name,history_date,history_version,history_change_type,history_change
0,94297.0,Время решения 3ЛП ФАКТ,09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223
1,94297.0,Время решения (ФАКТ),09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223
2,94297.0,Исполнитель,7/13/23 11:07,1.0,FIELD_CHANGED,user409017mail@mail.com -> user408045mail@mail...
3,NaN,NaN,NaN,NaN,NaN,NaN
4,94297.0,Исполнитель,7/21/23 11:06,3.0,FIELD_CHANGED,user4080458@mail.com -> user4091341@mail.com
...,...,...,...,...,...,...
64174,5179881.0,Спринт,10/23/24 7:00,11.0,FIELD_CHANGED,Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...
64175,5179881.0,Спринт,10/23/24 7:00,11.0,FIELD_CHANGED,Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...
64176,5179881.0,Спринт,10/23/24 7:00,11.0,FIELD_CHANGED,Спринт 2024.4.1.NPP Shared Sprint -> Спринт 20...
64177,5179881.0,Резолюция,10/23/24 7:00,12.0,FIELD_CHANGED,<empty> -> Готово


In [150]:

# Разделение entity_ids в таблице sprints
sprints['entity_ids'] = sprints['entity_ids'].apply(lambda x: x.strip('{}').split(',') if pd.notnull(x) else [])
sprints_expanded = sprints.explode('entity_ids')
sprints_expanded['entity_id'] = sprints_expanded['entity_ids'].astype(float)

# Удаляем строки с некорректным типом в entity_id из history
history = history[history['entity_id'].apply(lambda x: isinstance(x, (int, float)))]

# Приведение entity_id в обоих таблицах к одному типу (float)
data['entity_id'] = data['entity_id'].astype(float)
history['entity_id'] = history['entity_id'].astype(float)

# Соединение data и sprints
data_sprints = pd.merge(data, sprints_expanded, on='entity_id', how='left')

# Соединение data_sprints с history
final_table = pd.merge(data_sprints, history, on='entity_id', how='left')

# Просмотр результата
print(final_table.head())

# Если требуется сохранение в файл
final_table.to_csv('merged_data.csv', index=False)

data = final_table.copy()

   entity_id                 area    type   status   state priority  \
0    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
1    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
2    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
3    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
4    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   

  ticket_number                                               name  \
0     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
1     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
2     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
3     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
4     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   

                  create_date created_by  ...  \
0  2023-03-16 16:59:00.000000      А. К.  ...   
1  2023-03-16 16:59:00.000000      А. К.  ...   
2  20

In [151]:
#Вторая метрика : Снято

# Условия для фильтрации задач
valid_statuses = ["Закрыто", "Выполнено"]
valid_resolutions = ["Отклонено", "Отменено инициатором", "Дубликат", "Отклонён исполнителем"]

# Инициализация метрик
daily_metrics = []

# Преобразование дат в DataFrame
data['history_date'] = pd.to_datetime(data['history_date'], errors='coerce')

# Получение уникальных спринтов
unique_sprints = data['sprint_name'].unique()

# Цикл по каждому спринту
for sprint_name in unique_sprints:
    # Фильтрация данных по текущему спринту
    sprint_data = data[data['sprint_name'] == sprint_name]

    # Определение диапазона дат спринта
    sprint_start_date = sprint_data['history_date'].min()
    sprint_end_date = sprint_data['history_date'].max()

    # Проход по дням спринта
    current_date = sprint_start_date
    while current_date <= sprint_end_date:
        # Фильтрация задач, относящихся к текущему дню и удовлетворяющих условиям
        tasks_cancelled_on_day = sprint_data[
            (sprint_data['status'].isin(valid_statuses)) &
            (sprint_data['resolution'].isin(valid_resolutions)) &
            (sprint_data['history_date'] <= current_date)
        ]

        # Суммирование estimation и деление на 3600
        daily_metric = tasks_cancelled_on_day['estimation'].sum() / 3600

        # Добавление метрики в список
        daily_metrics.append({
            'sprint_name': sprint_name,
            'date': current_date,
            'second_metric': round(daily_metric, 1)
        })

        # Переход к следующему дню
        current_date += pd.Timedelta(days=1)

# Создание DataFrame с ежедневными метриками
daily_metrics_df = pd.DataFrame(daily_metrics)


In [152]:
# Выбор конкретного спринта
selected_sprint_entity_ids = sprints.iloc[0]['entity_ids']  # Уже является списком

# Фильтрация задач, относящихся к выбранному спринту, со статусом "Создано"
tasks_in_sprint = data[
    (data['entity_id'].isin(selected_sprint_entity_ids)) &
    (data['status'] == 'Создано')
]

# Суммирование estimation и деление на 3600
first_metric_sum = tasks_in_sprint['estimation'].sum() / 3600  # Перевод в часы

# Создание отдельного столбца с рассчитанным показателем для каждой строки (если нужен общий результат - только sum)
data['first_metric'] = 0  # Инициализация
data.loc[
    data['entity_id'].isin(selected_sprint_entity_ids),
    'first_metric'
] = first_metric_sum

In [153]:
# Группируем задачи по sprintName и рассчитываем метрику "К выполнению"
def calculate_metric_per_sprint(df, sprint_name):
    # Фильтрация задач, относящихся к указанному спринту и имеющих статус "Создано"
    tasks_in_sprint = df[
        (df['sprint_name'] == sprint_name) &
        (df['status'] == 'Создано')
    ]
    # Суммирование estimation для задач спринта и деление на 3600
    metric = tasks_in_sprint['estimation'].sum() / 3600
    return metric

# Применяем расчёт метрики по каждому уникальному спринту
sprint_metrics = {}
for sprint_name in data['sprint_name'].unique():
    sprint_metrics[sprint_name] = calculate_metric_per_sprint(data, sprint_name)

# Добавляем результат в таблицу
daily_metrics_df['first_metrick'] = data['sprint_name'].map(sprint_metrics)


In [154]:
import pandas as pd

# Преобразуем столбец sprint_start_date и sprint_end_date в datetime
sprints['sprint_start_date'] = pd.to_datetime(sprints['sprint_start_date'], errors='coerce')
sprints['sprint_end_date'] = pd.to_datetime(sprints['sprint_end_date'], errors='coerce')

# Функция для разделения задач по времени относительно дня спринта
def split_tasks_by_day(df, current_date):
    # Преобразуем history_date в datetime
    df['history_date'] = pd.to_datetime(df['history_date'], errors='coerce')
    
    # Задачи до текущей даты
    early_tasks_df = df[
        (df['history_date'].isna() | (df['history_date'] <= current_date)) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    # Задачи, добавленные до текущей даты
    added_tasks_df = df[
        (df['history_date'] > current_date) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    return early_tasks_df, added_tasks_df

# Функция для подсчета backlog_change для определённого дня спринта
def calculate_daily_backlog_change(data, sprint, current_date):
    # Разделяем задачи на те, что существовали до текущей даты, и те, что добавились
    early_tasks_df, added_tasks_df = split_tasks_by_day(data, current_date)

    # Считаем сумму оценок
    early_sum = early_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы
    added_sum = added_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы

    # Исправление расчета для случаев, когда early_sum == 0
    if early_sum > 0:
        backlog_change_pct = (added_sum * 100) / early_sum
    elif added_sum > 0:
        backlog_change_pct = 100.0  # Весь бэклог был добавлен поздно
    else:
        backlog_change_pct = 0.0  # Нет задач в бэклоге

    return round(backlog_change_pct, 1)

# Рассчитываем backlog_change для каждого дня в каждом спринте
daily_backlog_metrics = []
for _, sprint in sprints.iterrows():
    sprint_name = sprint['sprint_name']
    sprint_start_date = sprint['sprint_start_date']
    sprint_end_date = sprint['sprint_end_date']
    
    # Перебор дней спринта
    current_date = sprint_start_date
    while current_date <= sprint_end_date:
        backlog_change_pct = calculate_daily_backlog_change(data, sprint, current_date)
        daily_backlog_metrics.append({
            'sprint_name': sprint_name,
            'day': current_date,
            'backlog_change_percentage': backlog_change_pct
        })
        current_date += pd.Timedelta(days=1)  # Переходим к следующему дню

# Создаем итоговый DataFrame с ежедневными метриками
daily_backlog_metrics_df = pd.DataFrame(daily_backlog_metrics)


In [155]:
merged_df = pd.merge(
    daily_backlog_metrics_df,
    daily_metrics_df,
    on="sprint_name",        # Замените "date" на ваш общий столбец
    how="right"       # Тип соединения: 'inner', 'left', 'right', 'outer'
)

# Отобразим результат
print(merged_df.head())

                         sprint_name                 day  \
0  Спринт 2024.3.5.NPP Shared Sprint 2024-08-28 19:00:00   
1  Спринт 2024.3.5.NPP Shared Sprint 2024-08-29 19:00:00   
2  Спринт 2024.3.5.NPP Shared Sprint 2024-08-30 19:00:00   
3  Спринт 2024.3.5.NPP Shared Sprint 2024-08-31 19:00:00   
4  Спринт 2024.3.5.NPP Shared Sprint 2024-09-01 19:00:00   

   backlog_change_percentage                date  second_metric  first_metrick  
0                       33.6 2023-12-12 09:06:00            0.0         3056.0  
1                       33.6 2023-12-12 09:06:00            0.0         3056.0  
2                       33.6 2023-12-12 09:06:00            0.0         3056.0  
3                       33.6 2023-12-12 09:06:00            0.0         3056.0  
4                       33.6 2023-12-12 09:06:00            0.0         3056.0  


In [156]:
merged_df.shape

(34790, 6)